Google 에서도 sentencepiece 라는 이름으로 Word Piece Model package 를 공개하였습니다. 설치는 pip install 로 가능합니다. 실습 환경에서의 버전은 `sentencepiece==0.1.8` 입니다.

```
pip install sentencepiece
```

학습 데이터는 빈 칸이 포함되지 않은 문서 집합입니다. 우리의 실습 데이터에는 정규화 과정에서 한문으로만 이뤄진 기사의 텍스트가 지워졌기 때문에 빈 줄이 포함되어 있습니다. 이 빈 줄을 제거하여 `spm_input.txt` 파일을 만듭니다.

In [1]:
import config
import sentencepiece as spm
from navernews_10days import get_news_paths
from soynlp.utils import DoublespaceLineCorpus

path = get_news_paths(date='2016-10-20', tokenize=None)
corpus = DoublespaceLineCorpus(path, iter_sent=True, num_doc=1000)

input_file = 'spm_input.txt'
with open(input_file, 'w', encoding='utf-8') as f:
    for sent in corpus:
        f.write('{}\n'.format(sent))

Sentencepiece 는 C++ 로 구현되어 있으며, subprocess 를 이용하여 파이썬에서 스크립트를 실행시키는 방식입니다. Input 은 반드시 텍스트 파일이어야 합니다. command message 는 input, prefix, vocab_size 를 입력해야 합니다.

IPython notebook 에서는 파이썬 스크립트에서 오류가 날 경우 커널이 그대로 죽습니다. 파이썬 인터프리터에서 직접 작업하면 코드의 오류를 볼 수 있습니다. vocab_size 가 지나치게 작게 설정되면 모델이 학습을 하다 오류를 발생시킵니다. 당황하지 마시고 vocab_size 를 키우면 됩니다. 우리의 실습 예시에서는 vocab_size 1000 으로 설정하면 오류가 발생합니다. 2000 으로 키우면 문제없이 작동합니다.

In [2]:
templates = '--input={} --model_prefix={} --vocab_size={}'

vocab_size = 2000
prefix = '2016-10-20-news'
cmd = templates.format(input_file, prefix, vocab_size)

spm.SentencePieceTrainer.Train(cmd)

True

학습이 끝나면 `2016-10-20-news` 라는 prefix 를 지니는 .model 파일과 .vocab 파일이 만들어집니다.

In [3]:
ls

2016-10-20-news.model  day1_load_dataset_with_soynlp.ipynb    spm_input.txt
2016-10-20-news.vocab  day1_sentencepiece.ipynb               userdic.txt
config.py              day1_soynlp_word_noun_tokenizer.ipynb
day1_KoNLPy.ipynb      __pycache__/


학습된 모델을 이용하려면 .model 을 로딩합니다.

In [4]:
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(prefix))

True

문장을 입력하면 subword list 로 출력할 수 있습니다.

In [5]:
sp.EncodeAsPieces('오늘의 연합뉴스 기사입니다')

['▁오', '늘', '의', '▁연합뉴스', '▁기', '사', '입', '니', '다']

혹은 subword idx list 로 출력할 수도 있습니다.

In [6]:
sp.EncodeAsIds('오늘의 연합뉴스 기사입니다')

[203, 870, 7, 23, 171, 26, 327, 186, 25]

우리가 설정한 2000 개의 vocabulary 가 학습되었습니다.

In [7]:
with open('{}.vocab'.format(prefix), encoding='utf-8') as f:
    vocabs = [doc.strip() for doc in f]

print('num of vocabs = {}'.format(len(vocabs)))

num of vocabs = 2000


Vocabulary 에는 unknown, 문장의 시작, 문장의 끝, 단어의 앞부분 등 네 가지의 special token 이 있으며, 그 외에는 `을`, `이`, `의`, `는` 과 같은 조사들이 뒤따릅니다. `_기자`, `_재배포` 와 같은 도메인에서 자주 이용되는 단어들도 포함되어 있습니다. 

In [8]:
for vocab in vocabs:
    print(vocab)

<unk>	0
<s>	0
</s>	0
▁	-2.3141
을	-3.95619
이	-4.13792
는	-4.17755
의	-4.2608
에	-4.27495
를	-4.47744
은	-4.56576
가	-4.56862
한	-4.77574
고	-4.7846
도	-4.86284
에서	-4.93985
일	-4.98635
로	-5.02946
기	-5.11131
과	-5.1414
지	-5.15338
인	-5.24579
해	-5.31501
▁연합뉴스	-5.3214
▁20	-5.33494
다	-5.397
사	-5.41258
▁있다	-5.42713
리	-5.45189
으로	-5.45349
▁이	-5.47303
자	-5.52116
장	-5.52249
시	-5.58215
▁등	-5.59042
▁수	-5.61918
원	-5.62017
했다	-5.6344
대	-5.63529
정	-5.66812
하는	-5.74145
하고	-5.76434
와	-5.78018
년	-5.79992
▁서울	-5.82273
▁전	-5.8357
할	-5.84419
수	-5.85506
주	-5.85524
▁고	-5.89186
▁2	-5.92829
나	-5.95039
▁기자	-5.95057
상	-5.96083
전	-5.96911
스	-5.9739
▁3	-5.97818
▁1	-6.00681
만	-6.01671
어	-6.02466
성	-6.07467
부	-6.08663
▁10	-6.10234
제	-6.1093
서	-6.1269
천	-6.13931
구	-6.14533
화	-6.19967
면	-6.20454
들	-6.21878
하	-6.21914
▁김	-6.22977
회	-6.24134
명	-6.24962
게	-6.25369
선	-6.25562
진	-6.28571
월	-6.29294
▁2016	-6.29878
며	-6.30346
적	-6.3347
산	-6.35707
▁것	-6.36179
치	-6.37269
▁부	-6.37915
동	-6.38136
계	-6.39012
소	-6.39247
조	-6.39716
드	-6.40723
▁